In [ ]:
# Run setup_imports to enable importing from src/
%run ./setup_imports.py

In [ ]:
from qonscious.constraints.packed_chsh import PackedCHSHTest
from qonscious.policies import MinimumAcceptableValue
from qonscious.core.executor import run_conditionally
from qonscious.core.aer_sampler_adapter import AerSamplerAdapter

In [ ]:
passing_policy = MinimumAcceptableValue(1.5)
failing_policy = MinimumAcceptableValue(2.9)

In [ ]:
def on_pass(backend, introspection):
    print("CHSH test passed!")
    print(f"Score: {introspection['CHSH_score']:.3f}")
    return introspection

def on_fail(backend, introspection):
    print("CHSH test failed.")
    print(f"Score: {introspection['CHSH_score']:.3f}")
    return introspection

In [ ]:
backend = AerSamplerAdapter()
constraint = PackedCHSHTest(passing_policy)

print("Running CHSH test with passing threshold...\n")
run_conditionally(
    backend_adapter=backend,
    constraint=constraint,
    on_pass=on_pass,
    on_fail=on_fail,
    shots=2048
)


In [ ]:
constraint = PackedCHSHTest(failing_policy)

print("\nRunning CHSH test with failing threshold...\n")
run_conditionally(
    backend_adapter=backend,
    constraint=constraint,
    on_pass=on_pass,
    on_fail=on_fail,
    shots=2048
)